In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from matplotlib import pyplot as plt
import numpy as np
from data import *
import pandas as pd

In [ ]:
sales_train = pd.read_csv('data/sales_train.csv')
items = pd.read_csv('data/items.csv')
item_categories = pd.read_csv('data/item_categories.csv')
shops = pd.read_csv('data/shops.csv')

In [ ]:
merged_data = pd.merge(sales_train, items, on='item_id')
merged_data = pd.merge(merged_data, item_categories, on='item_category_id')
merged_data = pd.merge(merged_data, shops, on='shop_id')
shop_ids = merged_data['shop_id'].unique()

for shop_id in shop_ids:
    shop_data = merged_data[merged_data['shop_id'] == shop_id]


In [ ]:
X = shop_data[['item_id', 'item_category_id', 'date_block_num']]  # Features specific to each shop
y = shop_data['item_cnt_day']  # Target variable
scaler = StandardScaler()
X_norm = scaler.fit_transform(X)
print(f"Peak to Peak range by column in Raw        X:{np.ptp(X,axis=0)}")   
print(f"Peak to Peak range by column in Normalized X:{np.ptp(X_norm,axis=0)}")

In [ ]:
sgdr = SGDRegressor(max_iter=1000)
sgdr.fit(X_norm, y)

In [ ]:
b_norm = sgdr.intercept_
w_norm = sgdr.coef_

In [ ]:
y_pred_sgd = sgdr.predict(X_norm)
# make a prediction using w,b. 
y_pred = np.dot(X_norm, w_norm) + b_norm  
print(f"prediction using np.dot() and sgdr.predict match: {(y_pred == y_pred_sgd).all()}")

print(f"Prediction on training set:\n{y_pred[:4]}" )
print(f"Target values \n{y[:4]}")

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X['date_block_num'], y, color='blue', label='Actual')
plt.scatter(X['date_block_num'], y_pred_sgd, color='red', label='Predicted')
plt.title(f'Shop {shop_id} Actual vs Predicted Sales')
plt.xlabel('Date Block Number')
plt.ylabel('Item Count')
plt.legend()
plt.grid(True)
plt.show()